In [12]:
import requests

ENV = 'fch-yhb24'
url = 'https://api.weixin.qq.com/cgi-bin/token'
params = {
    'grant_type' : 'client_credential',
    'appid': '************',
    'secret': '************',
}
SQL_DIR = 'kjs_sql/'

r = requests.get(url, params= params)
if (r.status_code == 200):
    access_token = r.json()
    access_token.pop('expires_in')
    print(access_token)
else:
    print('Error Code: ' + str(r.status_code))

{'access_token': '26_Mu7rvnzQtRkDXiTbQ7GN9w-6RdCjKWYjqJjMJpvtVAfJ8tB5Y-wlX-dXHaTRgqxZVRN1EWpNVNtCi5uIyA-4_PUqDlFHs-SQP4k6pfanAALt4wQ2o4VmtOegv9IYDWhABABGZ'}


In [54]:
import json
import sys
from requests_toolbelt.multipart.encoder import MultipartEncoder


def upload_file(access_token, local_fp, remote_fp= None):
    url = 'https://api.weixin.qq.com/tcb/uploadfile'
    if remote_fp == None:
            remote_fp = local_fp
    post_data = {
        'env': ENV,
        'path': remote_fp,
    }
    r = requests.post(url, params= access_token, json=post_data)
    response = r.json()
    if response['errcode'] == 0:
        url = response['url']
        post_data = {
            'key': remote_fp,
            'Signature': response['authorization'],
            'x-cos-security-token': response['token'],
            'x-cos-meta-fileid': response['cos_file_id'],
        }
        with open(local_fp, 'rb') as f:
            post_data['file'] = f.read()
        m = MultipartEncoder(fields= post_data)
        headers = {
            'Content-type': m.content_type
        }
        r = requests.post(url, data= m, headers= headers)
        if r.text == '':
            sys.stdout.write('Flie ' + local_fp + ' uploaded.\r')
            return
    sys.stderr.write('Fail to upload flie ' + local_fp + ' .\n')
    
# upload_file(access_token, 'kjs_sql/002.json')

def import_database(access_token, collection_name, file_path, file_type=1):
    url = 'https://api.weixin.qq.com/tcb/databasemigrateimport'
    post_data = {
        'env': ENV,
        'collection_name': collection_name,
        'file_path': file_path,
        'file_type': file_type, # 1 for json, 2 for csv
        'stop_on_error': True,
        'conflict_mode': 2,     # 1 for insert, 2 for upsert
    }
    r = requests.post(url, json=post_data, params=access_token)
    response = r.json()
    if response['errcode'] == 0:
        sys.stdout.write('File ' + file_path + ' was successfully import to collection ' + collection_name + ' .\r')
    else:
        sys.stderr.write('Fail to import file ' + file_path + ' to collection ' + collection_name + ' .\n')
    
# file_path = SQL_DIR + '002.json'
# collection_name = 'test'
# upload_file(access_token, file_path)
# collection_clear(access_token, collection_name)
# collection_add(access_token, collection_name)
# import_database(access_token, 'test', file_path)


def main():
    building_dict = {}
    with open(SQL_DIR + 'upload_list.json', 'r', encoding='utf-8') as f:
        building_dict = json.loads(f.read())
    for building in building_dict:
        building_id, building_name = building['id'], building['name']
        file_path = SQL_DIR + building_id + '.json'
        upload_file(access_token, file_path)
        import_database(access_token, building_id, file_path)
    sys.stdout.write('\n----------------END----------------')
main()

In [12]:
def collection_add(access_token, collection_name):
    url = 'https://api.weixin.qq.com/tcb/databasecollectionadd'
    post_data = {
        'env': 'fch-yhb24',
        'collection_name': collection_name,
    }
    r = requests.post(url, params = access_token, json = post_data)
    if r.json()['errcode'] == 0:
        print('collection ' + collection_name + ' added.')
    else:
        print('fail to addcollection ' + collection_name + ' .')
        
def collection_del(access_token, collection_name):
    url = 'https://api.weixin.qq.com/tcb/databasecollectiondelete'
    post_data = {
        'env': ENV,
        'collection_name': collection_name,
    }
    r = requests.post(url, json= post_data, params = access_token)
    if r.json()['errcode'] == 0:
        print('collection ' + collection_name + ' deleted.')
    else:
        print('fail to delete collection ' + collection_name + ' .')

def batch_download(access_token, remote_fp):
    url = 'https://api.weixin.qq.com/tcb/batchdownloadfile'
    post_data = {
        'env': ENV,
        'file_list': [
            {
                'fileid': 'cloud://fch-yhb24.6663-fch-yhb24-1256334811/' + remote_fp,
                'max_age': 7200
            }
        ]
    }
    post_data = json.dumps(post_data)
    r = requests.post(url, params = access_token, data = post_data)
    response = json.loads(r.text)
    if response['errcode'] != 0 :
        print(r.text)
    else:
        file_list = response['file_list']
        for ele in file_list:
            print(ele['download_url'])

# batch_download(access_token, 'my-image.png')

In [21]:
def collection_clear(access_token, collection_name):
    url = 'https://api.weixin.qq.com/tcb/databasedelete'
    post_data = {
        'env': ENV,
        'query': 'db.collection(\"' + collection_name + '\").where({}).remove()'
    }
    r = requests.post(url, json= post_data, params= access_token)
    if r.json()['errcode'] == 0:
        print('collection ' + collection_name + ' cleared.')
    else:
        print('fail to clear collection ' + collection_name + ' .')
        print(r.json())

collection_clear(access_token, 'test')

fail to clear collection test .
{'errcode': -605101, 'errmsg': "SymanticError: no action found, did you forget to call 'get' (or another action) ?  hint: [j8YKlA08702162]"}


In [7]:
def invoke_cloud_function(access_token, ):
    url = 'https://api.weixin.qq.com/tcb/invokecloudfunction'
    access_token['env'] = ENV
    access_token['name'] = 'add'
    post_data = {
        'a': 1,
        'b': 2,
    }
    r =requests.post(url, params = access_token, data = json.dumps(post_data))
    print(r.request.body)
    print(r.json()['resp_data'])
    
invoke_cloud_function(access_token)

{"a": 1, "b": 2}
{"sum":3}
